In [11]:
%%writefile sample_tf_code.py

#import tensorflow as tf

def train_step(images, labels, mnist_model):
  loss_history = []
  with tf.GradientTape() as tape:
    logits = mnist_model(images, training=True)

    # Add asserts to check the shape of the output.
    tf.debugging.assert_equal(logits.shape, (32, 10))

    loss_value = loss_object(labels, logits)

  loss_history.append(loss_value.numpy().mean())
  grads = tape.gradient(loss_value, mnist_model.trainable_variables)
  optimizer.apply_gradients(zip(grads, mnist_model.trainable_variables))
  return np.mean(loss_history)


def train(epochs, dataset):

  mnist_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,[3,3], activation='relu', # convoutional layer
                         input_shape=(None, None, 1)),
    tf.keras.layers.Conv2D(16,[3,3], activation='relu'), # convoutional layer
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10) # output layer
  ])

  optimizer = tf.keras.optimizers.Adam()
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
  tot_loss = []
  for epoch in range(epochs):
    for (batch, (images, labels)) in enumerate(dataset):
      b_loss = train_step(images, labels, mnist_model)
      tot_loss.append(b_loss)
    #print ('Epoch {} finished'.format(epoch))
  return np.mean(tot_loss)

(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()

dataset = tf.data.Dataset.from_tensor_slices((tf.cast(mnist_images[...,tf.newaxis]/255, tf.float32), tf.cast(mnist_labels,tf.int64)))

dataset = dataset.shuffle(1000).batch(32)
final_loss = train(1, dataset)
print(final_loss)

Overwriting sample_tf_code.py


In [12]:
import os
import random
import bioblend
from bioblend.galaxy import GalaxyInstance
from bioblend.galaxy import histories
from time import sleep

server='http://127.0.0.1:9090/'
key = '49aec9e3e881e3235c601147b8e353f2'
file_name = "sample_tf_code.py"


gi = GalaxyInstance(server, key=key)
history = histories.HistoryClient(gi)

rnd_int = random.randint(1, 10000000)

print(rnd_int)

new_history = history.create_history(str(rnd_int))
print(new_history)
uploaded_dataset = gi.tools.upload_file(file_name, new_history["id"])
sleep(20)

2814543
{'annotation': None, 'contents_url': '/api/histories/7b55dbb89df8f4e5/contents', 'create_time': '2021-08-11T16:56:30.433598', 'deleted': False, 'empty': True, 'genome_build': None, 'id': '7b55dbb89df8f4e5', 'importable': False, 'model_class': 'History', 'name': '2814543', 'published': False, 'purged': False, 'size': 0, 'slug': None, 'state': 'new', 'state_details': {'discarded': 0, 'empty': 0, 'error': 0, 'failed_metadata': 0, 'new': 0, 'ok': 0, 'paused': 0, 'queued': 0, 'running': 0, 'setting_metadata': 0, 'upload': 0}, 'state_ids': {'discarded': [], 'empty': [], 'error': [], 'failed_metadata': [], 'new': [], 'ok': [], 'paused': [], 'queued': [], 'running': [], 'setting_metadata': [], 'upload': []}, 'tags': [], 'update_time': '2021-08-11T16:56:30.000437', 'url': '/api/histories/7b55dbb89df8f4e5', 'user_id': '2891970512fa2d5a', 'username_and_slug': None}


In [13]:
uploaded_dataset

{'implicit_collections': [],
 'jobs': [{'create_time': '2021-08-11T16:56:30.692510',
   'exit_code': None,
   'galaxy_version': '21.05',
   'history_id': '7b55dbb89df8f4e5',
   'id': '683bc220e21425bb',
   'model_class': 'Job',
   'state': 'new',
   'tool_id': 'upload1',
   'update_time': '2021-08-11T16:56:30.712828'}],
 'output_collections': [],
 'outputs': [{'create_time': '2021-08-11T16:56:30.614628',
   'data_type': 'galaxy.datatypes.data.Data',
   'deleted': False,
   'file_ext': 'auto',
   'file_size': 0,
   'genome_build': '?',
   'hda_ldda': 'hda',
   'hid': 1,
   'history_content_type': 'dataset',
   'history_id': '7b55dbb89df8f4e5',
   'id': '683bc220e21425bb',
   'metadata_dbkey': '?',
   'misc_blurb': None,
   'misc_info': None,
   'model_class': 'HistoryDatasetAssociation',
   'name': 'sample_tf_code.py',
   'output_name': 'output0',
   'peek': None,
   'purged': False,
   'state': 'queued',
   'tags': [],
   'update_time': '2021-08-11T16:56:30.673526',
   'uuid': 'af50d31

In [14]:
tool_name = "run_jupyter_job"
hist_id = new_history["id"]
file_path = uploaded_dataset["outputs"][0]["id"]
tool_inputs = {"inputs": {"select_file": file_path}}
tool_run = gi.tools.run_tool(hist_id, tool_name, tool_inputs)

In [15]:
tool_run["jobs"][0]["state"]

'new'

In [10]:
import tensorflow as tf
import numpy as np


def train_step(images, labels, mnist_model):
  loss_history = []
  with tf.GradientTape() as tape:
    logits = mnist_model(images, training=True)

    # Add asserts to check the shape of the output.
    tf.debugging.assert_equal(logits.shape, (32, 10))

    loss_value = loss_object(labels, logits)

  loss_history.append(loss_value.numpy().mean())
  grads = tape.gradient(loss_value, mnist_model.trainable_variables)
  optimizer.apply_gradients(zip(grads, mnist_model.trainable_variables))
  return np.mean(loss_history)


def train(epochs, dataset):
      
  (mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()

  dataset = tf.data.Dataset.from_tensor_slices(
    (tf.cast(mnist_images[...,tf.newaxis]/255, tf.float32),
     tf.cast(mnist_labels,tf.int64)))

  dataset = dataset.shuffle(1000).batch(32)

  mnist_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,[3,3], activation='relu', # convoutional layer
                         input_shape=(None, None, 1)),
    tf.keras.layers.Conv2D(16,[3,3], activation='relu'), # convoutional layer
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10) # output layer
  ])

  optimizer = tf.keras.optimizers.Adam()
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
  tot_loss = []
  for epoch in range(epochs):
    for (batch, (images, labels)) in enumerate(dataset):
      b_loss = train_step(images, labels, mnist_model)
      tot_loss.append(b_loss)
    #print ('Epoch {} finished'.format(epoch))
  return np.mean(tot_loss)
    
final_loss = train(1, dataset)
print(final_loss)

KeyboardInterrupt: 